# Lab - 2 :
  # Task - 3 : Recommender System

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Mounting the drive to read the csv files
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Task 3 Recommender System (20 Points)


## 1. Download m1.zip file from the link (https://grouplens.org/datasets/movielens/1m/) (0 points)

Downloaded the required files

## 2. Load the movies and ratings data (1 points)

In [ ]:
# Loading movies data
movies = pd.read_csv('/content/drive/My Drive/movies.dat', delimiter='::', header=None, 
    names=['movie_id', 'title', 'genre'], engine='python', encoding='ISO-8859-1')

In [ ]:
# Peeking at the movies dataset
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# Loading ratings data
ratings = pd.read_csv('/content/drive/My Drive/ratings.dat', delimiter='::', header=None, 
    names=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')

In [ ]:
# Looking at few records of the ratings dataset
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## 3. What do you mean by Singular Value Decomposition (2 points)?


The Singular Value Decomposition (SVD) of a matrix is a factorization of that matrix into three matrices.<br> It has some interesting algebraic properties and conveys important geometrical and theoretical insights about linear transformations.<br>SVD will take a matrix as an input and decompose it into a product of three simpler matrices like below:
<div>
<img src='https://miro.medium.com/max/894/1*XNWUlrQJXGeoCDqUMd0iUA.png' width='300'>
</div>
Singular value decomposition  represents a dataset by eliminating the less important parts and generating an accurate approximation of the original dataset.<br> In machine learning, SVD is widely used as a data reduction(dimensionality reduction) method.<br>SVD can also be used in least squares linear regression, image compression, and denoising data.

## 4. What do you mean by Principal Component Analysis (2 points)?

Principal Component Analysis (PCA) is a technique of Feature Extraction and is one of the leading linear techniques of dimensionality reduction.<br>

This method performs a direct mapping of the data to a lesser dimensional space in a way that maximizes the variance of the data in the low-dimensional representation.<br>

In other words, PCA is a method of obtaining important variables (in form of components) from a large set of variables available in a data set.<br> It extracts low dimensional set of features by taking a projection of irrelevant dimensions from a high dimensional data set with a motive to capture as much information as possible.<br> With fewer variables obtained while minimizing the loss of information, visualization also becomes much more meaningful.<br> PCA is more useful when dealing with 3 or higher dimensional data.<br><br>
For example, let's say we have a dataset with 50 features.<br>
Since we have large dimension, we can have 50(50 - 1)/2 scatter plots i.e more than 1000 plots possible to analyze the variable relationship which is not an ideal way to analyze the dataset.<br> 

<br>The image below shows the transformation of a high dimensional data (3 dimension) to low dimensional data (2 dimension) using PCA. Not to forget, each resultant dimension is a linear combination of all the original features

<div>
<img src='https://raw.githubusercontent.com/ritchieng/machine-learning-stanford/master/w8_unsupervised_learning/unsupervisedlearning22.png' width=600>
</div>

Below are some of the key points about PCA :

1. A principal component is a linear combination of the original variables
2. Principal components are extracted in such a way that the first principal component explains maximum variance in the dataset
3. Then, secondly, principal component tries to explain the remaining variance in the dataset and is uncorrelated to the first principal component
4. Third principal component tries to explain the variance which is not explained by the first two principal components and so on.

## 5. Explain content-based vs collaborative recommendation (2 points)

Recommender systems are an advantageous alternative to search algorithms.<br> They help users discover items they might not have found otherwise and offer personalized products to the taste of the users.<br> For this reason, any large platform needs a recommendation system algorithm to make the user’s shopping more enjoyable by automating the search process, offering personalized items, and saving their time.<br><br>
There are 2 major types of recommender systems : 
* Collaborative  
  Collaborative methods for recommender systems are methods that are based solely on the past interactions recorded between users and items in order to produce new recommendations.
* Content based 
  Unlike collaborative methods that only rely on the user-item interactions, content based approaches use additional information about users and/or items.
  <br><br>
  Below are the differences between Content-based and Collaborative recommender systems:

CONTENT-BASED  | COLLABORATIVE 
-------------------|------------------
In content based filtering, we use properties of the objects<br> and link similar ones and show them.       | In collaborative filtering, we usually use data of what was in any way linked together<br> by an outside sorting entity (e.g. bought together by an online shopper)<br> and show them in an ordered list.
A content-based recommendation engine emphasizes on the content features.     | A collaborative recommendation engine emphasizes on the user preference.
In content-based filtering, a recommendation system uses<br> the content profiles which includes the content features.     | In collaborative filtering, a recommendation engine requires the user profiles to suggest relevant content.
The content-based recommendation systems are product features oriented<br> and hence don't have cold start problems.    |  The collaborative recommendation systems feed on the user ratings, reviews, thumbs ups & downs,<br> and other feedback on various products or services.<br> So, the products with no ratings or feedback can't be recommended to any user.<br> Neither a new user who hasn't given any reviews or ratings can get any recommendation by the collaborative recommendation engine.<br>** This is called the cold start problem.**
A content-based recommendation engine can provide more accurate<br> recommendations as it focuses on the features of the content the user likes.       | A collaborative recommendation engine doesn't always ensure precise recommendations<br> because the users with similar taste may not like the same products.
Example of Content filtering: we show all the books that have have<br> same author, same publisher, same genre and the most similar number<br> of pages as book.      | Example of Collaborative filtering: we analyse what books have been read by people that have read book A<br> and the ones with highest count comes on the top of the list.

## 6. Create m x u matrix with movies as row and users as column. Normalize the matrix. (2 points)

In [ ]:
# Creating the m x u matrix with rows as movies and columns as users
m_u_mat = np.ndarray(shape=(np.max(ratings.movie_id.values), np.max(ratings.user_id.values)), dtype=np.uint8)
m_u_mat[ratings.movie_id.values-1, ratings.user_id.values-1] = ratings.rating.values

In [ ]:
m_u_mat

array([[  5,   0,   0, ...,  39, 205,   3],
       [  0,   0,   0, ...,  39, 205,  65],
       [  0,   0,   0, ...,  41, 205,  65],
       ...,
       [ 71,   1,   0, ...,   0,   0,   0],
       [100,   1,   0, ...,   0,   0,   0],
       [102,   9,   0, ...,   0,   0,   0]], dtype=uint8)

In [ ]:
# Normalizing the matrix by subtracting the mean from the data
normalised_mat = m_u_mat - np.asarray([(np.mean(m_u_mat, 1))]).T

In [ ]:
normalised_mat

array([[-48.85082781, -53.85082781, -53.85082781, ..., -14.85082781,
        151.14917219, -50.85082781],
       [-63.70612583, -63.70612583, -63.70612583, ..., -24.70612583,
        141.29387417,   1.29387417],
       [-74.61854305, -74.61854305, -74.61854305, ..., -33.61854305,
        130.38145695,  -9.61854305],
       ...,
       [ 53.92466887, -16.07533113, -17.07533113, ..., -17.07533113,
        -17.07533113, -17.07533113],
       [ 83.57996689, -15.42003311, -16.42003311, ..., -16.42003311,
        -16.42003311, -16.42003311],
       [ 86.36324503,  -6.63675497, -15.63675497, ..., -15.63675497,
        -15.63675497, -15.63675497]])

## 7. Perform SVD to get U, S and V (2 points)

In [ ]:
# Performing Singular Value Decomposition to reduce m_u_matrix to its constituent parts

A = normalised_mat.T / np.sqrt(m_u_mat.shape[0] - 1)
U, S, V = np.linalg.svd(A)

In [ ]:
U

array([[-2.31247364e-02, -2.59410569e-02, -3.95600606e-03, ...,
        -1.66888417e-04,  4.00301106e-05,  2.51908945e-04],
       [-1.40180142e-02,  1.72413040e-02,  5.22678807e-03, ...,
         1.19862911e-02,  4.22999940e-06, -2.45610003e-03],
       [-2.67517670e-03,  9.07475904e-03,  5.43251882e-03, ...,
        -5.17886366e-03, -1.77639301e-02,  7.74766485e-03],
       ...,
       [ 6.90264864e-03,  5.30680194e-03, -3.32482486e-02, ...,
         6.75023147e-01, -1.21619664e-03, -1.79708008e-02],
       [ 1.98991087e-02, -1.37234592e-02,  8.67152798e-03, ...,
         3.52384802e-03,  5.26167812e-01, -1.86650230e-02],
       [-3.42998404e-03,  9.74060227e-03,  5.91537290e-03, ...,
         2.43980616e-03,  3.65412618e-03,  2.62930625e-01]])

In [ ]:
S

array([3.03082494e+03, 1.40080530e+03, 1.15212291e+03, ...,
       5.01351438e-03, 4.92420143e-03, 4.79885728e-03])

In [ ]:
V

array([[ 1.65797138e-02,  2.00167562e-02,  2.32160001e-02, ...,
        -1.21331440e-02, -1.16758393e-02, -1.09807116e-02],
       [-1.13086217e-02, -1.32282382e-02, -1.53958374e-02, ...,
        -2.67175480e-02, -2.57167618e-02, -2.42453575e-02],
       [ 1.74868690e-02,  1.69500080e-02,  2.58440634e-02, ...,
        -4.66612616e-03, -4.50187830e-03, -4.23996999e-03],
       ...,
       [-8.04743341e-06,  1.95847241e-06, -2.35466310e-06, ...,
        -1.82913224e-04, -2.24895066e-04, -6.11248586e-05],
       [-1.94633216e-06, -2.52628165e-06,  3.09984387e-06, ...,
         5.35696290e-04,  1.32441940e-04, -1.00886274e-03],
       [-9.92543655e-07,  2.92011660e-06,  2.05365950e-08, ...,
        -7.61012132e-04,  5.65587054e-04,  7.13174366e-04]])

## 8. Select top 50 components from V.T (1 point)

In [ ]:
V_T_50 = V.T[:, :50]
V_T_50

array([[ 0.01657971, -0.01130862,  0.01748687, ..., -0.00378709,
         0.06130739,  0.01384019],
       [ 0.02001676, -0.01322824,  0.01695001, ...,  0.03277592,
         0.0250785 , -0.03028389],
       [ 0.023216  , -0.01539584,  0.02584406, ...,  0.02127364,
        -0.04888445,  0.00064658],
       ...,
       [-0.01213314, -0.02671755, -0.00466613, ..., -0.03499617,
        -0.00580994, -0.01071   ],
       [-0.01167584, -0.02571676, -0.00450188, ...,  0.00583776,
         0.00179773,  0.00294526],
       [-0.01098071, -0.02424536, -0.00423997, ..., -0.00750311,
         0.00185942,  0.00160853]])

## 9. Calculate the covariance matrix for the entire dataset (from step 6) (1 point)

In [ ]:
# Calculating the covariance matrix from the normalised matrix
cov_matrix = np.cov(normalised_mat, bias = True)
cov_matrix

array([[ 5292.52625759,  2765.40732109,  2769.99226942, ...,
         -893.638266  ,  -857.88999175,  -808.28084168],
       [ 2765.40732109,  5198.7710883 ,  3394.93508508, ...,
        -1081.48498133, -1040.21712603,  -978.51767549],
       [ 2769.99226942,  3394.93508508,  6507.06839788, ...,
        -1266.68335051, -1219.27090128, -1146.28823122],
       ...,
       [ -893.638266  , -1081.48498133, -1266.68335051, ...,
         2548.73157688,  1713.87613989,  1657.42239677],
       [ -857.88999175, -1040.21712603, -1219.27090128, ...,
         1713.87613989,  2405.65883708,  1618.23651534],
       [ -808.28084168,  -978.51767549, -1146.28823122, ...,
         1657.42239677,  1618.23651534,  2239.45480801]])

## 10. Get the eigen vectors from the covariance matrix (1 point)

In [ ]:
# Getting the eigenvalues and eigenvectors
eigenValues, eigenVectors = np.linalg.eig(cov_matrix)

In [ ]:
# Printing the eigenvalues
eigenValues = pd.DataFrame(eigenValues)
eigenValues

,0
0,6.008856e+06
1,1.283588e+06
2,8.682958e+05
3,2.460462e+05
4,6.870694e+04
...,...
3947,3.534602e-04
3948,3.486686e-04
3949,3.447498e-04
3950,3.342457e-04


In [ ]:
# Printing the eigenvectors
eigenVectors = pd.DataFrame(eigenVectors)
eigenVectors

,0,1,2,3,4,5,6,7,8,9,...,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951
0,0.016580,0.011309,0.017487,-0.005465,0.055998,0.071538,0.013003,-0.026616,0.019395,-0.011843,...,-6.812534e-06,1.470535e-05,0.000002,-0.000004,-0.000012,-0.000008,-1.153939e-05,0.000007,0.000011,0.000001
1,0.020017,0.013228,0.016950,-0.038763,0.012661,0.025485,-0.031462,-0.034240,0.035876,0.002375,...,-2.089513e-05,-6.338772e-06,-0.000006,0.000021,0.000031,-0.000007,8.718360e-06,0.000026,-0.000021,0.000019
2,0.023216,0.015396,0.025844,0.010734,-0.061657,0.017369,-0.060711,-0.051591,-0.054897,0.011760,...,-6.431913e-06,-2.220918e-05,-0.000014,-0.000011,-0.000042,0.000002,-2.442568e-05,0.000003,0.000005,0.000015
3,0.024006,0.014882,0.022889,-0.001788,0.057812,-0.016078,-0.039556,-0.001432,0.055479,-0.010596,...,-1.923593e-05,6.656039e-07,-0.000017,0.000007,-0.000007,0.000004,8.126579e-07,-0.000007,-0.000007,-0.000007
4,0.023376,0.015203,0.024465,-0.000790,-0.040951,0.007638,-0.039445,-0.033404,-0.034590,0.011949,...,9.137227e-07,-2.697595e-06,-0.000007,-0.000007,-0.000007,0.000003,1.694435e-05,0.000015,0.000009,-0.000021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3947,-0.010180,0.022666,-0.004007,0.000639,0.000179,-0.001328,0.000006,0.000662,-0.000481,0.000562,...,-7.788610e-04,8.005938e-04,0.001809,0.000209,-0.000097,0.000627,6.455850e-04,-0.000437,-0.001644,-0.000197
3948,-0.011314,0.025107,-0.004412,0.000742,0.000054,-0.000835,0.000157,-0.000049,-0.000134,0.000359,...,-8.525155e-05,1.809619e-04,-0.000516,0.000949,-0.000430,0.001399,-6.295154e-04,-0.000763,-0.000802,-0.002021
3949,-0.012133,0.026718,-0.004666,0.000714,-0.000022,0.000032,-0.000033,-0.000125,-0.000011,-0.000062,...,-8.684699e-04,1.056881e-03,0.001029,0.000507,0.000322,0.001515,-2.674561e-04,-0.000675,0.000411,-0.000052
3950,-0.011676,0.025717,-0.004502,0.000696,0.000015,0.000098,0.000023,0.000027,0.000075,0.000167,...,2.265522e-03,1.787609e-04,-0.000430,0.000249,0.002503,-0.000520,1.590558e-04,0.000424,0.000657,0.000423


## 11. Get the top 50 eigen vectors using eigen values (1 point)

In [ ]:
# Using the PCA to find out 50 components 
pca = PCA(n_components=50)
result_pca = pca.fit_transform(cov_matrix)
result_pca

array([[ 7.25629770e+04,  1.58342947e+04,  4.21624389e+03, ...,
        -3.30908141e+02, -4.32040280e+01, -3.60586303e+01],
       [ 9.30575124e+04,  1.67316178e+04,  8.42098396e+02, ...,
        -1.74319934e+02,  1.43672328e+02,  6.08449279e+01],
       [ 1.12147345e+05,  2.52734087e+04,  3.48866347e+02, ...,
         1.16597460e+02, -1.77405688e+02, -1.10251210e+02],
       ...,
       [-1.00836065e+05,  3.26869654e+03, -1.32095718e+04, ...,
        -9.22438917e+01,  3.89434039e+01,  6.04971269e+01],
       [-9.80251070e+04,  2.92894411e+03, -1.21120499e+04, ...,
         6.46047941e+00, -1.30851528e+01, -2.41174876e+01],
       [-9.37555455e+04,  2.44801815e+03, -1.04981503e+04, ...,
        -2.51465912e+01, -6.75946199e+01,  6.61584082e+00]])

## 12. Using cosine similarity find 10 closest movies using the 50 components from SVD (step 8) (2 points)

In [ ]:
#Function to calculate the cosine similarity (sorting by most similar and returning the top N)
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 # Movie id starts from 1 in the dataset
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

# Function to print top N similar movies
def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.movie_id == movie_id].title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])

In [ ]:
# Choosing the movie recommendation for movie_id = 153
movie_id = 153

In [ ]:
# Calculating the top indexes from the top_cosine_similarity
indexes = top_cosine_similarity(V_T_50, movie_id, 10)

# Printing the top 10 similar movies
print_similar_movies(movies, movie_id, indexes)

Recommendations for Batman Forever (1995): 

Batman Forever (1995)
Cliffhanger (1993)
Maverick (1994)
Last Action Hero (1993)
Waterworld (1995)
Beverly Hills Cop III (1994)
Three Musketeers, The (1993)
Under Siege 2: Dark Territory (1995)
Rumble in the Bronx (1995)
Richie Rich (1994)


## 13. Using cosine similarity find 10 closest movies using the 50 components from PCA (step 11) (2 points)

In [ ]:
# Calculating the top indexes from the top_cosine_similarity using the 50 components from the PCA
indexes_pca = top_cosine_similarity(result_pca, movie_id, 10)

# Printing the top 10 similar movies using PCA
print_similar_movies(movies, movie_id, indexes_pca)

Recommendations for Batman Forever (1995): 

Batman Forever (1995)
Hot Shots! Part Deux (1993)
Natural Born Killers (1994)
House Party 3 (1994)
Cliffhanger (1993)
Boomerang (1992)
Terminal Velocity (1994)
Bad Company (1995)
Casper (1995)
Circle of Friends (1995)


## 14. Compare the results of above two methods (1 point)

With the above results, we can see that the SVD and PCA method gave different recommendations for the movie with movie_id = 153.<br> But both don't have very large difference if we check the movie details as per the ratings.<br>Also, in this experiment, we found that PCA was faster than SVD. It was fast to find the top 50 components with higher variability then to decompose such large matrix.